# IBM Applied Data Science Capstone Course by Coursera
### Week 5 Final Report
**_Opening a New Chinese Restaurant in Los Angeles, California_**
- Build a dataframe of neighborhoods in Los Angeles, California by web scraping the data from Wikipedia page.
- Get the geographical coordinates of the neighborhoods.
- Obtain the venue data for the neighborhoods from Foursquare API.
- Explore and cluster the neighborhoods.
- Select the best cluster to open a new chinese restaurant.
***
### 1. Import libraries

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


### 2. Scrap data from Wikipedia page into a DataFrame

In [3]:
data = pd.read_html("https://en.wikipedia.org/wiki/List_of_cities_in_Los_Angeles_County,_California")
data = data[0]
data.head()

,City,Date incorporated,Population as of(2010 Census)
0,Agoura Hills,"December 8, 1982",20330
1,Alhambra,"July 11, 1903",83653
2,Arcadia,"August 5, 1903",56364
3,Artesia,"May 29, 1959",16522
4,Avalon,"June 26, 1913",3728


In [13]:
la_df = pd.DataFrame(data.iloc[:,0])
la_df.columns =['Neighborhood']
la_df.head()

,Neighborhood
0,Agoura Hills
1,Alhambra
2,Arcadia
3,Artesia
4,Avalon


In [14]:
# print the number of rows of the dataframe
la_df.shape

(88, 1)

### 3. Get the geographical coordinates

In [15]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Los Angeles County, California'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [16]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in la_df["Neighborhood"].tolist() ]

Status code Unknown from https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find: ERROR - HTTPSConnectionPool(host='geocode.arcgis.com', port=443): Read timed out. (read timeout=5.0)


In [17]:
coords

[[34.14611000000008, -118.77811999999994],
 [34.09370000000007, -118.12726999999995],
 [34.13614000000007, -118.03886999999997],
 [33.861140000000034, -118.07967999999994],
 [33.34411000000006, -118.32138999999995],
 [34.13361000000003, -117.90588999999994],
 [34.08526000000006, -117.96032999999994],
 [33.97977000000003, -118.18884999999995],
 [33.96998000000008, -118.14901999999995],
 [33.883260000000064, -118.12228999999996],
 [34.07346000000007, -118.40031999999997],
 [34.14531000000005, -117.97093999999998],
 [34.18182000000007, -118.30775999999997],
 [34.15233000000006, -118.69327999999996],
 [33.83161000000007, -118.26208999999994],
 [33.868660000000034, -118.06369999999998],
 [34.09641000000005, -117.71639999999996],
 [34.001190000000065, -118.15493999999995],
 [33.89569000000006, -118.22435999999999],
 [34.08743000000004, -117.88990999999999],
 [33.95983000000007, -118.17602],
 [34.02161000000007, -118.39620999999994],
 [34.000860000000046, -117.82906999999994],
 [33.9407700000

In [18]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [19]:
# merge the coordinates into the original dataframe
la_df['Latitude'] = df_coords['Latitude']
la_df['Longitude'] = df_coords['Longitude']

In [21]:
# check the neighborhoods and the coordinates
print(la_df.shape)
la_df.head()

(88, 3)


,Neighborhood,Latitude,Longitude
0,Agoura Hills,34.14611,-118.77812
1,Alhambra,34.09370,-118.12727
2,Arcadia,34.13614,-118.03887
3,Artesia,33.86114,-118.07968
4,Avalon,33.34411,-118.32139


In [22]:
# save the DataFrame as CSV file
la_df.to_csv("la_df.csv", index=False)

### 4. Create a map of Los Angeles County, California with neighborhoods superimposed on top

In [26]:
# get the coordinates of Los Angeles
address = 'Los Angeles County, California'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Los Angeles County, California {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Los Angeles County, California 34.3155072, -118.2096814.


In [27]:
# create map of Los Angeles using latitude and longitude values
la_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(la_df['Latitude'], la_df['Longitude'], la_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(la_map)  
    
map_kl

In [28]:
# save the map as HTML file
la_map.save('la_map.html')

### 5. Use the Foursquare API to explore the neighborhoods

In [29]:
# define Foursquare Credentials and Version
CLIENT_ID = 'your Foursquare Secret' # your Foursquare Secret
CLIENT_SECRET = 'your Foursquare Secret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IBWAX11WY1DFQB5LHPVFIFGLJ5OMCX1B4S3PWH2ZKKQVLKND
CLIENT_SECRET:ZVPVKI0Z2BNQVMF3N5FA5D3DZ1QFO1HNH5A4V4PE2F2BGQI5


**Now, let's get the top 100 venues that are within a radius of 2000 meters.**

In [67]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(la_df['Latitude'], la_df['Longitude'], la_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

KeyError: 'groups'

In [31]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(7316, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Agoura Hills,34.14611,-118.77812,Future Track Running Center,34.145819,-118.779251,Sporting Goods Shop
1,Agoura Hills,34.14611,-118.77812,Twisted Oak Tavern,34.145308,-118.778679,Gastropub
2,Agoura Hills,34.14611,-118.77812,Cafe Bizou,34.148410,-118.782587,French Restaurant
3,Agoura Hills,34.14611,-118.77812,Grissini Ristorante,34.145815,-118.778534,Italian Restaurant
4,Agoura Hills,34.14611,-118.77812,Pizza Nosh,34.148311,-118.782181,Pizza Place


**Let's check how many venues were returned for each neighorhood**

In [32]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Agoura Hills,97,97,97,97,97,97
Alhambra,100,100,100,100,100,100
Arcadia,100,100,100,100,100,100
Artesia,100,100,100,100,100,100
Avalon,71,71,71,71,71,71
Azusa,96,96,96,96,96,96
Baldwin Park,89,89,89,89,89,89
Bell,86,86,86,86,86,86
Bell Gardens,72,72,72,72,72,72


**Let's find out how many unique categories can be curated from all the returned venues**

In [33]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 343 uniques categories.


In [34]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Sporting Goods Shop', 'Gastropub', 'French Restaurant',
       'Italian Restaurant', 'Pizza Place', 'Park', 'Big Box Store',
       'Burger Joint', 'Fast Food Restaurant', 'Brewery',
       'Ice Cream Shop', 'Liquor Store', 'New American Restaurant',
       'Indian Restaurant', 'Deli / Bodega', 'American Restaurant',
       'Food Court', 'Warehouse Store', 'Grocery Store',
       'Mexican Restaurant', 'Restaurant', 'Sushi Restaurant',
       'Mediterranean Restaurant', 'Breakfast Spot', 'Gym', 'Café',
       'Steakhouse', 'Asian Restaurant', 'Arts & Crafts Store',
       'Paper / Office Supplies Store', 'Music Store', 'Sandwich Place',
       'Donut Shop', 'Bakery', 'Dumpling Restaurant',
       'Seafood Restaurant', 'Department Store', 'Multiplex',
       'Thai Restaurant', 'Chinese Restaurant', 'Clothing Store',
       'Juice Bar', 'Lounge', 'Laundry Service', 'Coffee Shop', 'ATM',
       'Pharmacy', 'Golf Course', 'Sports Bar', 'Convenience Store'],
      dtype=object)

### 6. Analyze Each Neighborhood

In [39]:
# creating dummy matirx or one hot encoding
la_dummy = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
la_dummy['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [la_dummy.columns[-1]] + list(la_dummy.columns[:-1])
la_dummy = la_dummy[fixed_columns]

print(la_dummy.shape)
la_dummy.head()

(7316, 344)


,Neighborhoods,ATM,Accessories Store,African Restaurant,Airport,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Business Service,Butcher,Cable Car,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Car Wash,Caribbean Restaurant,Casino,Castle,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,City Hall,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Bookstore,College Cafeteria,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Credit Union,Creperie,Cruise,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Dive Shop,Dive Spot,Doctor's Office,Dog Run,Donburi Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,Event Space,Fabric Shop,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Intersection,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Library,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Night Market,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoors & Recreation,Outlet Mall,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Club,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skatin

**Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [40]:
la_grouped = la_dummy.groupby(["Neighborhoods"]).mean().reset_index()

print(la_grouped.shape)
la_grouped

(87, 344)


,Neighborhoods,ATM,Accessories Store,African Restaurant,Airport,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Business Service,Butcher,Cable Car,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Car Wash,Caribbean Restaurant,Casino,Castle,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,City Hall,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Bookstore,College Cafeteria,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Credit Union,Creperie,Cruise,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Dive Shop,Dive Spot,Doctor's Office,Dog Run,Donburi Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,Event Space,Fabric Shop,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Intersection,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Library,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Night Market,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoors & Recreation,Outlet Mall,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Club,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skatin

In [43]:
len(la_grouped[la_grouped["Chinese Restaurant"] > 0])

66

**Create a new DataFrame for Chinese Restaurant data only**

In [44]:
la_cr = la_grouped[["Neighborhoods","Chinese Restaurant"]]

In [45]:
la_cr.head()

,Neighborhoods,Chinese Restaurant
0,Agoura Hills,0.041237
1,Alhambra,0.060000
2,Arcadia,0.020000
3,Artesia,0.040000
4,Avalon,0.000000


### 7. Cluster Neighborhoods
Run k-means to cluster the neighborhoods in Los Angeles into 3 clusters.

In [47]:
# set number of clusters
kclusters = 3

la_clustering = la_cr.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(la_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 1, 1, 0, 1, 0, 0])

In [48]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
la_merged = la_cr.copy()

# add clustering labels
la_merged["Cluster Labels"] = kmeans.labels_

In [49]:
la_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
la_merged.head()

,Neighborhood,Chinese Restaurant,Cluster Labels
0,Agoura Hills,0.041237,0
1,Alhambra,0.060000,0
2,Arcadia,0.020000,0
3,Artesia,0.040000,0
4,Avalon,0.000000,1


In [50]:
# merge la_cr_grouped with la_df to add latitude/longitude for each neighborhood
la_merged = la_merged.join(la_df.set_index("Neighborhood"), on="Neighborhood")

print(la_merged.shape)
la_merged.head() # check the last columns!

(87, 5)


,Neighborhood,Chinese Restaurant,Cluster Labels,Latitude,Longitude
0,Agoura Hills,0.041237,0,34.14611,-118.77812
1,Alhambra,0.060000,0,34.09370,-118.12727
2,Arcadia,0.020000,0,34.13614,-118.03887
3,Artesia,0.040000,0,33.86114,-118.07968
4,Avalon,0.000000,1,33.34411,-118.32139


In [52]:
# sort the results by Cluster Labels
print(la_merged.shape)
la_merged.sort_values(["Cluster Labels"], inplace=True)
la_merged.head()

(87, 5)


,Neighborhood,Chinese Restaurant,Cluster Labels,Latitude,Longitude
0,Agoura Hills,0.041237,0,34.14611,-118.77812
18,Compton,0.028986,0,33.89569,-118.22436
9,Bellflower,0.030000,0,33.88326,-118.12229
19,Covina,0.020000,0,34.08743,-117.88991
79,South Pasadena,0.020000,0,34.11580,-118.15213


**Finally, let's visualize the resulting clusters**

In [56]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=9)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(la_merged['Latitude'], la_merged['Longitude'], la_merged['Neighborhood'], la_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [57]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### 8. Examine Clusters

#### Cluster 0

In [64]:
print('Total number in cluster 0 is {}'.format(len(la_merged.loc[la_merged['Cluster Labels'] == 0])))
la_merged.loc[la_merged['Cluster Labels'] == 0]

Total number in cluster 0 is 35


,Neighborhood,Chinese Restaurant,Cluster Labels,Latitude,Longitude
0,Agoura Hills,0.041237,0,34.14611,-118.77812
18,Compton,0.028986,0,33.89569,-118.22436
9,Bellflower,0.030000,0,33.88326,-118.12229
19,Covina,0.020000,0,34.08743,-117.88991
79,South Pasadena,0.020000,0,34.11580,-118.15213
83,Walnut,0.034483,0,34.02693,-117.84050
77,South El Monte,0.032258,0,34.04644,-118.04326
54,Montebello,0.030000,0,34.01959,-118.11643
3,Artesia,0.040000,0,33.86114,-118.07968
6,Baldwin Park,0.022472,0,34.08526,-117.96033


#### Cluster 1

In [65]:
print('Total number in cluster 1 is {}'.format(len(la_merged.loc[la_merged['Cluster Labels'] == 1])))
la_merged.loc[la_merged['Cluster Labels'] == 1]

Total number in cluster 1 is 48


,Neighborhood,Chinese Restaurant,Cluster Labels,Latitude,Longitude
86,Westlake Village,0.012500,1,34.14987,-118.81875
53,Monrovia,0.010000,1,34.14783,-117.99878
29,Glendora,0.000000,1,34.13602,-117.86452
28,Glendale,0.000000,1,34.14633,-118.24864
23,Downey,0.010000,1,33.94077,-118.12768
24,Duarte,0.016129,1,34.13961,-117.97050
21,Culver City,0.000000,1,34.02161,-118.39621
32,Hermosa Beach,0.000000,1,33.86404,-118.39535
13,Calabasas,0.000000,1,34.15233,-118.69328
26,El Segundo,0.010753,1,33.91992,-118.41596


#### Cluster 2

In [66]:
print('Total number in cluster 2 is {}'.format(len(la_merged.loc[la_merged['Cluster Labels'] == 2])))
la_merged.loc[la_merged['Cluster Labels'] == 2]

Total number in cluster 2 is 4


,Neighborhood,Chinese Restaurant,Cluster Labels,Latitude,Longitude
55,Monterey Park,0.171053,2,34.05979,-118.12651
80,Temple City,0.080000,2,34.10728,-118.05668
67,Rosemead,0.110000,2,34.08075,-118.07562
70,San Gabriel,0.090000,2,34.09708,-118.10824


#### Observations:
As observations noted from the map, most of the chinese restaurants are concentrated in the central area of Los Angeles city, with the highest number in cluster 1 and moderate number in cluster 0. On the other hand, cluster 2 has very low number to no chinese restaurant in the neighbourhoods. This represents a great opportunity and high potential areas to open new chinese restaurants as there is very little to no competition from existing chinese restaurants.

Meanwhile, chinese restaurants in cluster 1 are likely suffering from intense competition due to oversupply and high concentration of chinese restaurants. From another perspective, the results also show that the oversupply of chinese restaurants mostly happened in the central area of the city, with the suburb area still have very few chinese restaurants. Therefore, this project recommends chinese restaurants owners or franchisee to capitalize on these findings to open new chinese restaurants in neighbourhoods in cluster 2 with little to no competition.

Lastly, they are advised to avoid neighbourhoods in cluster 1 which already have high concentration of chinese restaurants and suffering from intense competition